In [1]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from CNN import CNN
from tqdm import tqdm
import torch.nn as nn
from trainDataSet import trainDataSet
import matplotlib.pyplot as plt
import pandas as pd
torch.manual_seed(1)  # 使用随机化种子使神经网络的初始化每次都相同
from testDataSet import testDataSet
# 超参数
EPOCH = 5 # 训练整批数据的次数
BATCH_SIZE = 64
LR = 0.001  # 学习率

C:\Users\20230921a\.conda\envs\pymarl\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [58]:
correct_file_label=[]
def delete_error_img():
    """
    由于train数据集中图片噪音过多，现用官方数据训练完成的模型对一些标签错误的图片进行筛选，模型准确率已经在98%左右；
    如果模型预测的标签与实际标签不一致，则删除该图片
    """

    root_dir = './processed_data/train'  # 替换为数据集根目录路径
    dataset = trainDataSet(root_dir)
    train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
    # 加载模型

    cnn = CNN()
    # 如果模型已经训练过，确保加载模型权重
    #该模型是用mnist官方数据集训练完成的
    cnn.load_state_dict(torch.load('cnn2.pkl'))
    # 将模型设置为评估模式
    cnn.eval()
    correct_index=[]#需要被删除的下标
    for index, (data, label) in enumerate(tqdm(train_loader)):
        # 模型预测
        output = cnn(data)
        _, predicted = torch.max(output.data, 1)
        # 检查预测是否正确
        batch_start_index = index * train_loader.batch_size
        for idx, pred in enumerate(predicted):
            absolute_idx = batch_start_index + idx  # 计算在整个数据集中的索引
            if pred.item() == label[idx].item():
                correct_index.append(absolute_idx)
                correct_file_label.append(pred.item())
              
    print("删除噪音数据中........大概2min")
    dataset.get_correct_data(correct_index)
    dataset.set_labels(correct_file_label)
    # 创建一个新的 dataset
    new_dataset = train_loader.dataset
    return new_dataset


def train_for_new_model(train_loader):
    """
    dataloader是要训练的数据集
    通过cnn训练已经清理完成的数据集，来得到一个针对于该数据集的新模型。
    """
    cnn2 = CNN()
    # 优化器选择Adam
    optimizer = torch.optim.Adam(cnn2.parameters(), lr=LR)
    # 损失函数
    loss_func = nn.CrossEntropyLoss()  # 目标标签是one-hotted
    # 开始训练
    for epoch in range(EPOCH):
        for step, (b_x, b_y) in enumerate(tqdm(train_loader)):  # 分配batch data
            output = cnn2(b_x)  # 先将数据放到cnn中计算output
            loss = loss_func(output, b_y)  # 输出和真实标签的loss，二者位置不可颠倒
            optimizer.zero_grad()  # 清除之前学到的梯度的参数
            loss.backward()  # 反向传播，计算梯度
            optimizer.step()  # 应用梯度
    torch.save(cnn2.state_dict(), 'cnnLast.pkl')#保存新模型

In [59]:

newDataSet=delete_error_img()

100%|██████████| 938/938 [00:21<00:00, 43.45it/s]


删除噪音数据中........大概2min


AttributeError: 

In [49]:
len(newDataSet)

44702

In [62]:
newDataSet.set_labels(correct_file_label)
filename=newDataSet.data_files
labels=newDataSet.labels
correctFile=pd.DataFrame({"name":filename,"labels":labels,"correct_labels":correct_file_label}).to_csv("1.csv")

AttributeError: 

In [ ]:
# root_dir = './processed_data/train'  # 替换为数据集根目录路径
# dataset1 = trainDataSet(root_dir)
# train_loader = DataLoader(dataset1, batch_size=BATCH_SIZE, shuffle=True)
# print(len(dataset1))
# old_filename=train_loader.dataset.data_files
# old_labels=train_loader.dataset.labels
# pd.DataFrame({"name":old_filename,"labels":old_labels}).to_csv("hahah1.csv")

In [51]:
cnn = CNN()
# 如果模型已经训练过，确保加载模型权重
#该模型是用mnist官方数据集训练完成的
cnn.load_state_dict(torch.load('cnn2.pkl'))
# 将模型设置为评估模式
cnn.eval()


CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout(p=0.5, inplace=False)
  )
  (out): Linear(in_features=1568, out_features=10, bias=True)
)

In [ ]:
#看留下的数据集在原来的模型中准确率如何
correct_predictions = 0
total_samples = 0

pp=[]
la=[]
newloader=DataLoader(newDataSet,batch_size=BATCH_SIZE,shuffle=False)
with torch.no_grad():
    for data, labels in tqdm(newloader):
        outputs = cnn(data)
        _, predicted = torch.max(outputs, 1)
        total_samples += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()
        pp.append(predicted.cpu().numpy())
        la.append(labels.cpu().numpy())
accuracy = correct_predictions / total_samples
print(f'模型在数据集上的准确率：{accuracy * 100:.2f}%')


In [61]:
# correctFile['pred_lab']=pp
flattened_array1 = np.concatenate(pp)
flattened_array2 = np.concatenate(la)
# correctFile['correct_label']=flattened_array
pd.DataFrame(flattened_array1).to_csv("2.csv",index=False)
pd.DataFrame(flattened_array2).to_csv("3.csv",index=False)

In [ ]:
correct_predictions = 0
total_samples = 0
with torch.no_grad():
    for data, labels in tqdm(train_loader):
        outputs = cnn(data)
        _, predicted = torch.max(outputs, 1)
        total_samples += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()
accuracy = correct_predictions / total_samples
print(f'模型在数据集上的准确率：{accuracy * 100:.2f}%')